In [1]:

# Import required libraries and dependencies
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from pathlib import Path
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# Load the data into a Pandas DataFrame
df_census_income = pd.read_csv("adult.csv")

# Display sample data
df_census_income.head(10)


,Age,Workclass,Final Weight,Education,EducationNum,Marital Status,Occupation,Relationship,Race,Gender,Capital Gain,capital loss,Hours per Week,Native Country,Income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,White,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Black,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States,>50K
8,31,Private,45781,Masters,14,Never-married,Prof-specialty,Not-in-family,White,Female,14084,0,50,United-States,>50K
9,42,Private,159449,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,5178,0,40,United-States,>50K


In [3]:
pd.unique(df_census_income["Workclass"])

array([' State-gov', ' Self-emp-not-inc', ' Private', ' Federal-gov',
       ' Local-gov', ' ?', ' Self-emp-inc', ' Without-pay',
       ' Never-worked'], dtype=object)

In [4]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
df_census_income['Workclass'].value_counts()

,count
Workclass,
Private,22696
Self-emp-not-inc,2541
Local-gov,2093
?,1836
State-gov,1298
Self-emp-inc,1116
Federal-gov,960
Without-pay,14
Never-worked,7


In [5]:
df_census_income['Workclass'] = df_census_income['Workclass'].str.strip()
df_census_income['Workclass'] = df_census_income['Workclass'].str.lower()

In [6]:
# Filter out rows where 'Workclass' is 'never-worked' or 'without-pay'
df_census_income = df_census_income[
    ~df_census_income['Workclass'].isin(['never-worked', 'without-pay'])
]


In [7]:
df_census_income['Workclass'] = df_census_income['Workclass'].replace('?', 'Unknown')

In [8]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
df_census_income['Workclass'].value_counts()

,count
Workclass,
private,22696
self-emp-not-inc,2541
local-gov,2093
Unknown,1836
state-gov,1298
self-emp-inc,1116
federal-gov,960


In [9]:
df_census_income = df_census_income.drop(columns=['Education','Capital Gain','capital loss'])
df_census_income.rename(columns={"EducationNum": "Education Rank"}, inplace=True)


df_census_income.head()

,Age,Workclass,Final Weight,Education Rank,Marital Status,Occupation,Relationship,Race,Gender,Hours per Week,Native Country,Income
0,39,state-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
1,50,self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
2,38,private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
3,53,private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
4,28,private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K


In [10]:
df_census_income['Marital Status'].value_counts()


,count
Marital Status,
Married-civ-spouse,14967
Never-married,10674
Divorced,4442
Separated,1025
Widowed,992
Married-spouse-absent,417
Married-AF-spouse,23


In [11]:
df_census_income['Occupation'] = df_census_income['Occupation'].str.strip().replace('?', 'Unknown')
df_census_income['Occupation'].value_counts()


,count
Occupation,
Prof-specialty,4140
Craft-repair,4098
Exec-managerial,4066
Adm-clerical,3767
Sales,3650
Other-service,3294
Machine-op-inspct,2001
Unknown,1836
Transport-moving,1596


In [12]:
# Ensure 'Income' column is of type string
df_census_income['Income'] = df_census_income['Income'].astype(str).str.strip()

# Replace values
df_census_income['Income'] = df_census_income['Income'].replace({
    '>50K': '1',
    '<=50K': '0'
})

# Rename column
df_census_income.rename(columns={"Income": "Income Greater than 50k"}, inplace=True)

# Verify the result
print(df_census_income['Income Greater than 50k'].value_counts())


Income Greater than 50k
0    24699
1     7841
Name: count, dtype: int64


In [13]:
df_census_income['Native Country'].value_counts()

,count
Native Country,
United-States,29150
Mexico,643
?,583
Philippines,197
Germany,137
Canada,121
Puerto-Rico,114
El-Salvador,106
India,100


In [14]:
# Clean up any whitespace from the column
df_census_income['Native Country'] = df_census_income['Native Country'].str.strip()

# Replace all non-'United-States' values with 'Other'
df_census_income['Native Country'] = df_census_income['Native Country'].apply(
    lambda x: 'Other' if x != 'United-States' else x
)

df_census_income['Native Country'].value_counts()

,count
Native Country,
United-States,29150
Other,3390


In [15]:
df_census_income.head()

,Age,Workclass,Final Weight,Education Rank,Marital Status,Occupation,Relationship,Race,Gender,Hours per Week,Native Country,Income Greater than 50k
0,39,state-gov,77516,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,0
1,50,self-emp-not-inc,83311,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,0
2,38,private,215646,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,0
3,53,private,234721,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,0
4,28,private,338409,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Other,0


In [16]:
# Create a Spark session
spark = SparkSession.builder \
    .appName("CensusIncomeAnalysis") \
    .getOrCreate()

In [17]:
# Convert pandas DataFrame to PySpark DataFrame
spark_df = spark.createDataFrame(df_census_income)

In [18]:
# Extract Income column as a separate DataFrame
income_df = spark_df.select("Income Greater than 50k")

# Show the first few rows
income_df.show()

+-----------------------+
|Income Greater than 50k|
+-----------------------+
|                      0|
|                      0|
|                      0|
|                      0|
|                      0|
|                      0|
|                      0|
|                      1|
|                      1|
|                      1|
|                      1|
|                      1|
|                      0|
|                      0|
|                      1|
|                      0|
|                      0|
|                      0|
|                      0|
|                      1|
+-----------------------+
only showing top 20 rows



In [19]:
# Drop the Income column from the main DataFrame
features_df = spark_df.drop("Income Greater than 50k")

# Show the first few rows
features_df.show()

+---+----------------+------------+--------------+--------------------+-----------------+--------------+-------------------+-------+--------------+--------------+
|Age|       Workclass|Final Weight|Education Rank|      Marital Status|       Occupation|  Relationship|               Race| Gender|Hours per Week|Native Country|
+---+----------------+------------+--------------+--------------------+-----------------+--------------+-------------------+-------+--------------+--------------+
| 39|       state-gov|       77516|            13|       Never-married|     Adm-clerical| Not-in-family|              White|   Male|            40| United-States|
| 50|self-emp-not-inc|       83311|            13|  Married-civ-spouse|  Exec-managerial|       Husband|              White|   Male|            13| United-States|
| 38|         private|      215646|             9|            Divorced|Handlers-cleaners| Not-in-family|              White|   Male|            40| United-States|
| 53|         private|

In [20]:
# Convert features_df and income_df from PySpark to pandas
features_pd = features_df.toPandas()
income_pd = income_df.toPandas()

income_pd.head()

,Income Greater than 50k
0,0
1,0
2,0
3,0
4,0


In [21]:
# Encode the categorical variables using get_dummies
features_pd = pd.get_dummies(features_pd)
features_pd = features_pd.astype(int)
features_pd.head(20)

,Age,Final Weight,Education Rank,Hours per Week,Workclass_Unknown,Workclass_federal-gov,Workclass_local-gov,Workclass_private,Workclass_self-emp-inc,Workclass_self-emp-not-inc,...,Relationship_ Wife,Race_ Amer-Indian-Eskimo,Race_ Asian-Pac-Islander,Race_ Black,Race_ Other,Race_ White,Gender_ Female,Gender_ Male,Native Country_Other,Native Country_United-States
0,39,77516,13,40,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,1
1,50,83311,13,13,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,1
2,38,215646,9,40,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1
3,53,234721,7,40,0,0,0,1,0,0,...,0,0,0,1,0,0,0,1,0,1
4,28,338409,13,40,0,0,0,1,0,0,...,1,0,0,1,0,0,1,0,1,0
5,37,284582,14,40,0,0,0,1,0,0,...,1,0,0,0,0,1,1,0,0,1
6,49,160187,5,16,0,0,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
7,52,209642,9,45,0,0,0,0,0,1,...,0,0,0,0,0,1,0,1,0,1
8,31,45781,14,50,0,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,1
9,42,159449,13,40,0,0,0,1,0,0,...,0,0,0,0,0,1,0,1,0,1


In [22]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(features_pd, income_pd, random_state=24)

In [23]:
# Create the StandardScaler instance
scaler = StandardScaler()
# Fit the Standard Scaler with the training data
X_scaler = scaler.fit(X_train)
# Scale the training data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
LOGISTIC REGRESSION MODEL

In [24]:
classifier = LogisticRegression(solver='lbfgs', random_state=1)
classifier

LogisticRegression(random_state=1)

In [25]:
# Train the data
classifier.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=1)

In [26]:
# Predict outcomes for test data set
predictions = classifier.predict(X_test_scaled)

In [27]:
# Create and save the confusion matrix for the data
conf_matrix = confusion_matrix(y_test, predictions)

# Print the confusion matrix for the training data
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[5686  444]
 [ 879 1126]]


In [28]:
# Print the classification report for the model
# Create and save the testing classification report
testing_report = classification_report(y_test, predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.87      0.93      0.90      6130
           1       0.72      0.56      0.63      2005

    accuracy                           0.84      8135
   macro avg       0.79      0.74      0.76      8135
weighted avg       0.83      0.84      0.83      8135



In [29]:
classifier = LogisticRegression(solver='lbfgs', random_state=2)
classifier

LogisticRegression(random_state=2)

In [30]:
# Train the data
classifier.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=2)

In [31]:
# Create and save the confusion matrix for the data
conf_matrix = confusion_matrix(y_test, predictions)

# Print the confusion matrix for the training data
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[5686  444]
 [ 879 1126]]


In [32]:
# Print the classification report for the model
# Create and save the testing classification report
testing_report = classification_report(y_test, predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.87      0.93      0.90      6130
           1       0.72      0.56      0.63      2005

    accuracy                           0.84      8135
   macro avg       0.79      0.74      0.76      8135
weighted avg       0.83      0.84      0.83      8135



In [56]:
classifier = LogisticRegression(solver='lbfgs', random_state=7)
classifier

LogisticRegression(random_state=7)

In [57]:
# Train the data
classifier.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=7)

In [58]:
# Create and save the confusion matrix for the data
conf_matrix = confusion_matrix(y_test, predictions)

# Print the confusion matrix for the training data
print("Confusion Matrix:")
print(conf_matrix)

Confusion Matrix:
[[5686  444]
 [ 879 1126]]


In [59]:
# Print the classification report for the model
# Create and save the testing classification report
testing_report = classification_report(y_test, predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

           0       0.87      0.93      0.90      6130
           1       0.72      0.56      0.63      2005

    accuracy                           0.84      8135
   macro avg       0.79      0.74      0.76      8135
weighted avg       0.83      0.84      0.83      8135



In [ ]:
KNN MODEL

SyntaxError: invalid syntax (<ipython-input-9-814428b94f56>, line 1)

In [38]:
model = KNeighborsClassifier(n_neighbors=6, )

In [39]:
# Train the model
model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=6)

In [40]:
# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array(['0', '0', '1', ..., '0', '0', '1'], dtype=object)

In [41]:
# Print confusion matrix
confusion_matrix(y_pred,y_test)

array([[5709, 1053],
       [ 421,  952]])

In [45]:
# Print classification report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.93      0.84      0.89      6762
           1       0.47      0.69      0.56      1373

    accuracy                           0.82      8135
   macro avg       0.70      0.77      0.72      8135
weighted avg       0.85      0.82      0.83      8135



In [46]:

model = KNeighborsClassifier(n_neighbors=8, )

In [47]:

# Train the model
model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=8)

In [48]:


# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array(['0', '0', '1', ..., '0', '0', '1'], dtype=object)

In [49]:

# Print confusion matrix
confusion_matrix(y_pred,y_test)

array([[5676,  982],
       [ 454, 1023]])

In [50]:

# Print classification report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.93      0.85      0.89      6658
           1       0.51      0.69      0.59      1477

    accuracy                           0.82      8135
   macro avg       0.72      0.77      0.74      8135
weighted avg       0.85      0.82      0.83      8135



In [51]:

model = KNeighborsClassifier(n_neighbors=3, )

In [52]:

# Train the model
model.fit(X_train_scaled, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [53]:
# Create predictions
y_pred = model.predict(X_test_scaled)

# Review the predictions
y_pred

array(['0', '0', '1', ..., '0', '0', '1'], dtype=object)

In [54]:
# Print confusion matrix
confusion_matrix(y_pred,y_test)

array([[5445,  900],
       [ 685, 1105]])

In [55]:

# Print classification report
print(classification_report(y_pred,y_test))

              precision    recall  f1-score   support

           0       0.89      0.86      0.87      6345
           1       0.55      0.62      0.58      1790

    accuracy                           0.81      8135
   macro avg       0.72      0.74      0.73      8135
weighted avg       0.81      0.81      0.81      8135



SUMMARY OF RESULTS ^(i.e. postives vs. negatives)

RANDOM FOREST ATTEMPT 1 (Nodes = 1500)

In [ ]:
rf_model = RandomForestClassifier(n_estimators=1500, random_state=68)

In [ ]:
# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)

In [ ]:
# Making predictions using the testing data
predictions = rf_model.predict(X_test_scaled)

In [ ]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [ ]:

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))